In [7]:
from google.colab import drive
drive.mount('/drive')
import pandas as pd
import numpy as np

from collections import Counter
from collections import OrderedDict

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm

Mounted at /drive


In [8]:
import nltk #библиотека для анализа текста
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
nltk.download('punkt')
nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


- [Ноутбук-парсер](https://colab.research.google.com/drive/1TYIdqMCi_LVXlQTXb6NXl5bAaZRFcE3r?usp=sharing)

- [Ноутбук с преобразованием после парсинга и формированием итогового датасета](https://colab.research.google.com/drive/1gpbbtj6USRIrekerWTgq73R2GiW-Ep2A?usp=sharing)

- [используемый dataset(после парсинга_cырой)](https://drive.google.com/file/d/1LA0lVpvVLfxVklhas4LPx5ZA9caeZaO3/view?usp=sharing)

- [ноутбук с первычным преобразованием итогового датасета, заполнением пропусков](https://colab.research.google.com/drive/1zo5hGTqKU0O7tW8Um3n_YTZwhrQSI7GA?usp=sharing)

- [dataset после заполнения пропусков зарплата по среднему](https://drive.google.com/file/d/1wcI8J4GchwyCTDNScEQg7_5KG6NNvmGz/view?usp=sharing)

- [dataset после заполнения пропусков зарплата по медиане](https://drive.google.com/file/d/1iD1oEew_a91ydoF2teIJU1T9jQ5cZHd0/view?usp=sharing)



In [9]:
#это датасет слепленный из того, что спарсил и того, что был на kaggle/ в соледующих ячейках он значительно фильтруется
# df_origin = pd.read_csv('/drive/MyDrive/!SberDataScience/Курсовой проект/!main/dataset/final_after_fill_na.csv')

df_origin = pd.read_csv('/drive/MyDrive/!SberDataScience/Курсовой проект/!main/dataset/final_after_fill_na_salary_median.csv')

In [10]:
df_origin.iloc[:3,:].head(3)

,Employer,Name,Salary,From,To,Experience,Schedule,Skills,Description,Category,Position_level,Salary_median
0,Клевер Солюшнс,Backend Python Developer (middle/team lead),True,140000.0,200000.0,От 3 до 6 лет,Полный день,"Git,Redis,Django Framework,Базы данных,Python,...",Компания Qlever Solutions занимается разработк...,Python Develop,Middle,170000.0
1,FunBox,Python-разработчик,True,170000.0,210000.0,От 1 года до 3 лет,Полный день,"Python,Django,Git,Linux,Работа в команде",Вам предстоит участвовать в разработке сложных...,Python Develop,Middle,190000.0
2,Spark Equation,Data Scientist (Remote),True,120000.0,190000.0,От 3 до 6 лет,Удаленная работа,"Machine Learning,Python,Java,SQL,Data Mining,D...",Spark Equation is a software-enabled IT and st...,Data Science,other,155000.0


In [11]:
df_origin.Category.value_counts()

Python Develop    1482
Data Analysis     1051
Data Science       775
Data Engineer      702
Name: Category, dtype: int64

In [12]:
df_origin['Category'] = df_origin['Category'].replace({'Data Science':1, 'Data Analysis':2, 'Data Engineer':3, 'Python Develop':4})
df_origin.shape

(4010, 12)

In [13]:
origin_columns = df_origin.columns.to_list()
cat_columns = ['Experience', 'Schedule']
text_columns = ['Name', 'Skills', 'Description']


In [14]:
df = pd.get_dummies(df_origin[cat_columns])
df_origin = pd.concat([df_origin, df], axis = 1)
df_origin.shape

(4010, 20)

In [15]:
dummi_columns = df.columns.to_list()

In [16]:
X_train, X_val, y_train, y_val = train_test_split(df_origin[text_columns+dummi_columns], df_origin['Category'], test_size=0.3, random_state = 33)

In [17]:
russian_stop_words = stopwords.words('russian')
snowball = SnowballStemmer(language='russian')


def tokenize_sentence(sentence):
  tokens = word_tokenize(sentence, language = 'russian')
  tokens = sentence.split(' ')
  tokens = [i for i in tokens if i not in [string.punctuation, '...',"'",':',';',',','.','-','+']]
  tokens = [i for i in tokens if i not in russian_stop_words]
  tokens = [snowball.stem(i) for i in tokens]

  return tokens



In [18]:
from sklearn.compose import make_column_transformer, ColumnTransformer

# vectorize_columns = ColumnTransformer(transformers = 
#     [
#      ('tokenizer_Skills', TfidfVectorizer(tokenizer = lambda x: tokenize_sentence(x)), 'Skills'),
#      ('tokenizer_Description', TfidfVectorizer(tokenizer = lambda x: tokenize_sentence(x)), 'Description'),
#      ('tokenizer_Name', TfidfVectorizer(tokenizer = lambda x: tokenize_sentence(x)), 'Name'),
#     ]
# )

# vectorize_columns = ColumnTransformer(transformers = 
#     [
#      ('tokenizer_Skills', TfidfVectorizer(max_features = 100, min_df=5, max_df=0.7, ngram_range=(1, 1)), 'Skills'),
#      ('tokenizer_Description', TfidfVectorizer(max_features = 100, min_df=5, max_df=0.7, ngram_range=(2, 3)), 'Description'),
#      ('tokenizer_Name', TfidfVectorizer(max_features = 100, min_df=5, max_df=0.7, ngram_range=(1, 3)), 'Name'),
#     ]
# )

# vectorize_columns = ColumnTransformer(transformers = 
#     [
#      ('tokenizer_Skills', TfidfVectorizer(tokenizer = lambda x: tokenize_sentence(x), max_features = 100, min_df=5, max_df=0.7, ngram_range=(1, 1)), 'Skills'),
#      ('tokenizer_Description', TfidfVectorizer(tokenizer = lambda x: tokenize_sentence(x), max_features = 100, min_df=5, max_df=0.7, ngram_range=(2, 3)), 'Description'),
#      ('tokenizer_Name', TfidfVectorizer(tokenizer = lambda x: tokenize_sentence(x), max_features = 100, min_df=5, max_df=0.7, ngram_range=(1, 3)), 'Name'),
#     ]
# )


vectorize_columns = ColumnTransformer(transformers = 
    [
     ('tokenizer_Skills', TfidfVectorizer(tokenizer = tokenize_sentence, max_features = 100, min_df=5, max_df=0.7, ngram_range=(1, 1)), 'Skills'),
     ('tokenizer_Description', TfidfVectorizer(tokenizer = tokenize_sentence, max_features = 100, min_df=5, max_df=0.7, ngram_range=(2, 3)), 'Description'),
     ('tokenizer_Name', TfidfVectorizer(tokenizer = tokenize_sentence, max_features = 100, min_df=5, max_df=0.7, ngram_range=(1, 3)), 'Name'),
    ]
)


In [19]:
tokenized_columns_train = pd.DataFrame.sparse.from_spmatrix(vectorize_columns.fit_transform(X_train))
tokenized_columns_train.shape

(2807, 300)

In [20]:
tokenized_columns_val = pd.DataFrame.sparse.from_spmatrix(vectorize_columns.transform(X_val))
tokenized_columns_val.shape

(1203, 300)

In [21]:
X_train_new = pd.concat([tokenized_columns_train, X_train.iloc[:,3:].reset_index(drop = True)], axis = 1)
X_train_new.shape 

(2807, 308)

In [22]:
model = KNeighborsClassifier()
model.fit(X_train_new, y_train)

KNeighborsClassifier()

In [23]:
X_val_new = pd.concat([tokenized_columns_val, X_val.iloc[:,3:].reset_index(drop = True)], axis = 1)
X_val_new.shape 


(1203, 308)

In [24]:
y_pred = model.predict(X_val_new)
y_pred

array([2, 4, 2, ..., 1, 3, 4])

In [25]:
accuracy_score(y_val, y_pred)

0.9201995012468828

In [26]:
pip install catboost

In [27]:
import catboost as cb

In [36]:
y_train.value_counts()

4    1027
2     737
1     557
3     486
Name: Category, dtype: int64

In [35]:
catboost = cb.CatBoostClassifier( n_estimators=500,
                                # classes_count=4,
                                # learning_rate = 0.02898,   
                                depth = 5,
                                l2_leaf_reg = 5.3,
                                # grow_policy = 'SymmetricTree',
                                loss_function='MultiClass',
                                random_state=46,
                                # bootstrap_type='Bernoulli',
                                # subsample = 0.66,
                                )
catboost.fit(X_train_new, y_train,
              verbose = 100
              )
y_pred = catboost.predict(X_val_new)

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

0:	learn: 1.3292756	total: 164ms	remaining: 1m 21s
100:	learn: 0.2576952	total: 7.83s	remaining: 31s
200:	learn: 0.1863919	total: 12.9s	remaining: 19.2s
300:	learn: 0.1602051	total: 20.2s	remaining: 13.4s
400:	learn: 0.1472003	total: 31.6s	remaining: 7.8s
499:	learn: 0.1374911	total: 40.1s	remaining: 0us


In [37]:
accuracy_score(y_val, y_pred)

0.9559434746467166